## 주식 가격 (동종사)

In [17]:
from pykrx import stock
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

# 동종사 코드
stock_codes = [
    '004690',   # 삼천리
    '017390',   # 서울도시가스
    '034590',   # 인천도시가스
    '267290'    # 경동도시가스
]

In [ ]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for stock_code in stock_codes:
    # 저장된 데이터의 마지막 일자 가져오기
    query = f'SELECT tr_date FROM stock_price where stock_code = {stock_code} ORDER BY tr_date DESC LIMIT 1'
    cur.execute(query)
    row = cur.fetchone()
    last_date = row[0]     # 저장된 데이터의 마지막 일자
    # KRX에서 데이터를 가져올 시작일과 종료일 설정
    start_date = myutil.get_next_day(last_date)
    end_date = myutil.get_previous_day()
    # 특정기간 특정 종목의 OHLCV
    ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)
    df_ohlcv = ohlcv.reset_index()
    # 날짜 형식 변환 => 문자열 yyyymmdd
    df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
    df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')
    print(f'주식 종목 {stock_code}, {start_date}에서 {end_date}까지의 가격정보 {df_ohlcv.shape[0]}건의 데이터 저장')
    for row in df_ohlcv.itertuples():
        sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                                values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
    conn.commit()

conn.close()